In [1]:
MAIN_FOLDER = 'dataset'
ANNOTATED_DATASET_FOLDER = 'annotated-images' 
BOXED_DATASET_FOLDER = 'images-with-boxes'
CROPPED_IMAGES_FOLDER = 'cropped-coa'

# Get the COA with their coordinates

In [2]:
import pandas as pd
import json

lables = {}
not_annotated = []
with open('dataset/2021-03-19_classification-labeling-results.json') as json_file:
    data = json.load(json_file)
    for task in data:
        im_id= task['image']
        lables[im_id] = []
        for label_result in task['selected_labels']:   
            try:
                sample = {'description': label_result['coa_description'], 
                          'coordinates': label_result['coa_coordinates']
                         }
                
                lables[im_id].append(sample)
                
            except KeyError as e:
                  not_annotated.append(im_id)
            

print('samples without coordinates: ', len(not_annotated))
print('samples with coordinates: ', len(lables))


samples without coordinates:  452
samples with coordinates:  599


# Crop COA from images

In [5]:
from PIL import Image

# The coordinates of the COAs are all normalized
# Using those values directly without normalization doesn't work 
def denormalize(left, upper, right, lower, size):
    w, l = size
        
    return left*w, upper*l, right*w, lower*l    

def get_coa_coordinates(coa, size):
    left = coa['coordinates']['topL']['x']
    upper = coa['coordinates']['topL']['y']
    right = coa['coordinates']['botR']['x']
    lower = coa['coordinates']['botR']['y']

    return denormalize(left, upper, right, lower, size)

def crop_image(img_id, img_data):
    img = Image.open(MAIN_FOLDER + '/'+  img_id)
    for coa in img_data:
        desc = coa['description']
        left, upper, right, lower = get_coa_coordinates(coa,img.size)
        img2 = img.crop((left, upper, right, lower))
        img2.save(MAIN_FOLDER + '/' + CROPPED_IMAGES_FOLDER + '/' + desc + ".png")


In [6]:
for img_id, img_data in lables.items():
    crop_image(img_id, img_data)

